In [20]:
import warnings
warnings.filterwarnings('ignore')

# Метод наискорейшего спуска

## Объяснение метода

In [21]:
import sympy as sym
import numpy as np

### Дано

variables = x1, x2 = sym.symbols('x1 x2')

# функция из методички
f = x1**2 + x1*x2 + 2*x2**2 + x1 - x2

x0 = np.array([0, 0])
eps = 0.1

f

x1**2 + x1*x2 + x1 + 2*x2**2 - x2

In [22]:
N = len(variables)

a = sym.symbols('a')

# составляем переменные s_i для каждого x_i
s_variables = sym.symbols(' '.join( [f's{i}' for i in range(1, N + 1)] ) )

s_variables

(s1, s2)

In [23]:
# [(x_i, a*s_i + x_i)] - лист кортежей с заменой
replacement = [(variables[i], variables[i] + a * s_variables[i]) for i in range(N)]

replacement

[(x1, a*s1 + x1), (x2, a*s2 + x2)]

In [24]:
# градиент функции по всем переменным
gradients_f = [sym.diff(f, variable) for variable in variables]

display(*gradients_f)

2*x1 + x2 + 1

x1 + 4*x2 - 1

In [25]:
# заменяем переменные в градиенте
gradients_f = [f.subs(replacement) for f in gradients_f]

display(*gradients_f)

2*a*s1 + a*s2 + 2*x1 + x2 + 1

a*s1 + 4*a*s2 + x1 + 4*x2 - 1

In [26]:
# домножаем градиент по каждой переменной на s_i
expr = sum([gradients_f[i] * s_variables[i] for i in range(N)])
expr

s1*(2*a*s1 + a*s2 + 2*x1 + x2 + 1) + s2*(a*s1 + 4*a*s2 + x1 + 4*x2 - 1)

In [28]:
# выражаем a
solution = sym.solve(expr, a)[0]
solution

(-2*s1*x1 - s1*x2 - s1 - s2*x1 - 4*s2*x2 + s2)/(2*(s1**2 + s1*s2 + 2*s2**2))

Получаем функцию, зависящую от:

* начальной точки с координатами x1..xn 

* градиента функции начальной точки s1..sn

## Наискорейший спуск

In [29]:
import sympy as sym
import numpy as np

norm = np.linalg.norm

def get_alfa_from_x_and_gradient(variables, f):
    N = len(variables)

    a = sym.symbols('a')

    # составляем переменные s_i для каждого x_i
    s_variables = sym.symbols(' '.join( [f's{i}' for i in range(1, N + 1)] ) )

    # [(x_i, a*s_i + x_i)] - лист кортежей с заменой
    replacement = [(variables[i], variables[i] + a * s_variables[i]) for i in range(N)]

    # градиент функции по всем переменным
    gradients_f = [sym.diff(f, variable) for variable in variables]

    # заменяем переменные в градиенте
    gradients_f = [f.subs(replacement) for f in gradients_f]

    # домножаем градиент по каждой переменной на s_i
    expr = sum([gradients_f[i] * s_variables[i] for i in range(N)])
    
    # выражаем a
    solution = sym.solve(expr, a)[0]

    return sym.lambdify(variables + s_variables, solution)

def get_func_gradient_from_point(variables, f, x0):
    # градиент функции по всем переменным
    gradients_f = [sym.diff(f, variable) for variable in variables]

    # превращение из символьного представления в функцию
    lambd_gradients_f = [
        sym.lambdify(variables, gradient_f) for gradient_f in gradients_f]
    
    grad_f_x = np.array([f(*x0) for f in lambd_gradients_f])

    return np.where(np.abs(grad_f_x) > 0.0001, grad_f_x, 0)

def steepest_descent(
    variables: list[sym.Symbol],
    f: sym.Add,
    x0: np.ndarray,
    eps: float
):
    x1 = x0

    alfa = get_alfa_from_x_and_gradient(variables, f)

    for _ in range(1000):
        grad_f_x = get_func_gradient_from_point(variables, f, x1)
        
        alfa_ = alfa(*x1, *-grad_f_x)

        x1 = x0 + alfa_ * -grad_f_x

        if norm(x1 - x0) < eps:
            return x1
        x0 = x1

In [30]:
### Дано
variables = x1, x2 = sym.symbols('x1 x2')
# функция из методички
f = x1**2 + x1*x2 + 2*x2**2 + x1 - x2

x0 = np.array([0, 0])
eps = 0.1

steepest_descent(variables, f, x0, eps)

array([-0.6875,  0.4375])

In [4]:
# функция 7 варианта
f = 10 - (x1 - 3) * sym.exp(-x1 + 3) - (x2 - 2) * sym.exp(-x1 + 2)

print(steepest_descent(variables, f, np.array([0,0]), 0.1))

None
